In [5]:
import numpy as np
import BioSimSpace as BSS

In [3]:
# Paths

In [10]:
# Load input files
ligand_1 = BSS.IO.readMolecules("gtv.mol2")[0]
ligand_2 = BSS.IO.readMolecules("ykg.mol2")[0]
# print(ligand_2)

In [11]:
ligand_2

<BioSimSpace.Molecule: nAtoms=28, nResidues=1>

In [8]:
protein = BSS.IO.readMolecules("6d18_clean.pdb")[0]
protein

<BioSimSpace.Molecule: nAtoms=4017, nResidues=270>

# Parameterising input molecules

Try using SMILES strings to directly parameterise

In [14]:
# ligand_2 = BSS.Parameters.gaff2("Cc1cc2c(c(c1)Br)C(=CC(=O)O2)CP(=O)(O)O").getMolecule()

In [16]:
# ligand_1 = BSS.Parameters.gaff2("Cc1cc(c2c(c1)OC(=O)C=C2CP(=O)(O)O)C").getMolecule()

In [31]:
view = BSS.Notebook.View(ligand_1)
view.system()

ThemeManager()

NGLWidget(gui_style='ngl')

In [32]:
view = BSS.Notebook.View(ligand_2)
view.system()

NGLWidget(gui_style='ngl')

# Try parameterising .mol2 files


In [9]:
ligand_1 = BSS.Parameters.gaff2(ligand_1).getMolecule()

In [13]:
ligand_2 = BSS.Parameters.gaff2(ligand_2).getMolecule()

# NOTE

Before the below step, run `pdb4amber` to first remove hydrogens and then to add all missing atoms. Details on Notion.

In [14]:
protein = BSS.Parameters.parameterise(protein, forcefield="ff14SB").getMolecule()

In [33]:
view = BSS.Notebook.View(protein)
view.system()

NGLWidget(gui_style='ngl')

# Alignment and merging

To transform two ligands, they need to be well aligned. In BSS, we can do this with the function `Align`. It uses a Maximum Common Substructure (MCS) search, which finds mappings between atom indices in the two molecules. 

`BSS.Align.matchAtoms()` matches the atoms with MCS. It finds mappings between atom indices in molecule 1 and molecule 2. 

`BSS.Align.rmsdAlign()` uses the above mapping to actually align the atoms from molecule 1 to molecule 2. 

In [15]:
atom_mapping = BSS.Align.matchAtoms(ligand_1, ligand_2)
aligned_ligand_1 = BSS.Align.rmsdAlign(ligand_1, ligand_2, atom_mapping)

Now we have to create a 'merged' molecule, i.e. a molecule that we can transform in a way such that the endpoints are both input ligands:

In [16]:
merged = BSS.Align.merge(ligand_1, ligand_2)

IncompatibleError: Cannot determine 'forcefield' of 'molecule0'!

Add the merged ligand to the protein to create the system:

In [35]:
system = merged + protein

# Solvation

We will have two environments:
1. Each ligand (`merged`) in the solvent, **not bound** to the protein, call this `unbound`
2. Each ligand (`system`) in the solvent, **bound** to the protein, call this `bound`.


In [29]:
box_dimensions =  3 * [10 * BSS.Units.Length.nanometer] 
unbound = BSS.Solvent.tip3p(molecule=merged, box=box_dimensions)
bound = BSS.Solvent.tip3p(molecule=system, box=box_dimensions)

# FEP Protocol

Now we can write the protocol and the set up files and soon finally run the FEP.  

In [17]:
BSS.FreeEnergy.engines()

['Somd', 'Gromacs']

In [30]:
protocol = BSS.Protocol.FreeEnergy()
free_energy_ub = BSS.FreeEnergy.Relative(unbound, protocol, engine="Somd", work_dir="output/unbound/")
free_energy_b = BSS.FreeEnergy.Relative(bound, protocol, engine="Somd", work_dir="output/bound/")

# Setup folders etc for running on GPUs

See Notion.